In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("CreditRiskAnalysis").getOrCreate()

data = [
    Row(CustomerID=1, FirstName="John", LastName="Doe", TotalDebt=15000.0, GrossIncome=50000.0, 
        CreditUsed=3000.0, TotalCredit=10000.0, PaymentHistoryScore=85, NumberOfDefaults=1),
    Row(CustomerID=2, FirstName="Jane", LastName="Smith", TotalDebt=25000.0, GrossIncome=60000.0, 
        CreditUsed=8000.0, TotalCredit=20000.0, PaymentHistoryScore=90, NumberOfDefaults=0),
    Row(CustomerID=3, FirstName="Alice", LastName="Johnson", TotalDebt=30000.0, GrossIncome=45000.0, 
        CreditUsed=10000.0, TotalCredit=15000.0, PaymentHistoryScore=75, NumberOfDefaults=2),
    Row(CustomerID=4, FirstName="Bob", LastName="Brown", TotalDebt=12000.0, GrossIncome=48000.0, 
        CreditUsed=4000.0, TotalCredit=12000.0, PaymentHistoryScore=80, NumberOfDefaults=1),
    Row(CustomerID=5, FirstName="Charlie", LastName="Davis", TotalDebt=18000.0, GrossIncome=52000.0, 
        CreditUsed=5000.0, TotalCredit=15000.0, PaymentHistoryScore=70, NumberOfDefaults=2),
    Row(CustomerID=6, FirstName="David", LastName="Wilson", TotalDebt=22000.0, GrossIncome=55000.0, 
        CreditUsed=7000.0, TotalCredit=20000.0, PaymentHistoryScore=88, NumberOfDefaults=0),
    Row(CustomerID=7, FirstName="Eva", LastName="Clark", TotalDebt=9000.0, GrossIncome=47000.0, 
        CreditUsed=2500.0, TotalCredit=10000.0, PaymentHistoryScore=92, NumberOfDefaults=0),
    Row(CustomerID=8, FirstName="Frank", LastName="Miller", TotalDebt=30000.0, GrossIncome=49000.0, 
        CreditUsed=9500.0, TotalCredit=20000.0, PaymentHistoryScore=65, NumberOfDefaults=3),
    Row(CustomerID=9, FirstName="Grace", LastName="Taylor", TotalDebt=16000.0, GrossIncome=51000.0, 
        CreditUsed=2000.0, TotalCredit=12000.0, PaymentHistoryScore=78, NumberOfDefaults=1),
    Row(CustomerID=10, FirstName="Hannah", LastName="Anderson", TotalDebt=17000.0, GrossIncome=53000.0, 
        CreditUsed=6000.0, TotalCredit=13000.0, PaymentHistoryScore=82, NumberOfDefaults=1),
    Row(CustomerID=11, FirstName="Ian", LastName="Thomas", TotalDebt=20000.0, GrossIncome=54000.0, 
        CreditUsed=7500.0, TotalCredit=15000.0, PaymentHistoryScore=79, NumberOfDefaults=2),
    Row(CustomerID=12, FirstName="Jack", LastName="Jackson", TotalDebt=24000.0, GrossIncome=60000.0, 
        CreditUsed=8500.0, TotalCredit=18000.0, PaymentHistoryScore=91, NumberOfDefaults=0),
    Row(CustomerID=13, FirstName="Kathy", LastName="White", TotalDebt=11000.0, GrossIncome=46000.0, 
        CreditUsed=3500.0, TotalCredit=11000.0, PaymentHistoryScore=84, NumberOfDefaults=1),
    Row(CustomerID=14, FirstName="Laura", LastName="Harris", TotalDebt=13000.0, GrossIncome=51000.0, 
        CreditUsed=2000.0, TotalCredit=12000.0, PaymentHistoryScore=80, NumberOfDefaults=1),
    Row(CustomerID=15, FirstName="Mike", LastName="Martin", TotalDebt=19000.0, GrossIncome=52000.0, 
        CreditUsed=6000.0, TotalCredit=14000.0, PaymentHistoryScore=73, NumberOfDefaults=2),
    Row(CustomerID=16, FirstName="Nina", LastName="Thompson", TotalDebt=21000.0, GrossIncome=57000.0, 
        CreditUsed=5000.0, TotalCredit=15000.0, PaymentHistoryScore=89, NumberOfDefaults=0),
    Row(CustomerID=17, FirstName="Oscar", LastName="Garcia", TotalDebt=26000.0, GrossIncome=68000.0, 
        CreditUsed=10000.0, TotalCredit=25000.0, PaymentHistoryScore=72, NumberOfDefaults=3),
    Row(CustomerID=18, FirstName="Paula", LastName="Martinez", TotalDebt=13000.0, GrossIncome=44000.0, 
        CreditUsed=3000.0, TotalCredit=10000.0, PaymentHistoryScore=87, NumberOfDefaults=0),
    Row(CustomerID=19, FirstName="Quincy", LastName="Lopez", TotalDebt=27000.0, GrossIncome=72000.0, 
        CreditUsed=15000.0, TotalCredit=30000.0, PaymentHistoryScore=68, NumberOfDefaults=4),
    Row(CustomerID=20, FirstName="Rachel", LastName="Wilson", TotalDebt=16000.0, GrossIncome=46000.0, 
        CreditUsed=4000.0, TotalCredit=11000.0, PaymentHistoryScore=90, NumberOfDefaults=1)
]

In [0]:
df = spark.createDataFrame(data)
df.show(truncate=False)

In [0]:
# Calculate Debt-to-Income ratio
df = df.withColumn("DTI", col("TotalDebt") / col("GrossIncome"))

# Calculate Credit Utilization Ratio
df = df.withColumn("CUR", col("CreditUsed") / col("TotalCredit"))

In [0]:
# Step 7: Define risk categories based on DTI and CUR
def risk_category(dti, cur):
    if dti > 0.4 or cur > 0.3:
        return "High Risk"
    elif dti > 0.2 or cur > 0.2:
        return "Medium Risk"
    else:
        return "Low Risk"

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

risk_udf = udf(risk_category, StringType())
df = df.withColumn("RiskCategory", risk_udf(col("DTI"), col("CUR")))
df.show(truncate=False)

In [0]:
df.select("CustomerID", "FirstName", "LastName", "DTI", "CUR", "RiskCategory").show(truncate=False)